In [1]:
### THIS IS COMPARSION BETWEEN PYSPARK AND PANDAS LOAD TIME###

import pandas as pd
import numpy as np
import time
from pyspark.sql import SparkSession    
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType


In [13]:
df_pandas = pd.read_json("D:\Project\Spark_project\combined_file.txt",lines=True)

<>:1: SyntaxWarning: invalid escape sequence '\P'
<>:1: SyntaxWarning: invalid escape sequence '\P'
C:\Users\raksh\AppData\Local\Temp\ipykernel_29832\2043049915.py:1: SyntaxWarning: invalid escape sequence '\P'
  df_pandas = pd.read_json("D:\Project\Spark_project\combined_file.txt",lines=True)


## we can notice that pandas has taken around 1 min to load file of size 1GB 


In [14]:
print(df_pandas.shape)
df_pandas.head()

(1829647, 3)


,type,dt,payload
0,new_adsb,2025-01-31 00:00:00.330843,"{'hex': 'ab35d3', 'flight': 'DAL136 ', 'alt_b..."
1,new_adsb,2025-01-31 00:00:00.336607,"{'hex': 'c03f37', 'flight': 'WJA598 ', 'alt_b..."
2,new_adsb,2025-01-31 00:00:00.336872,"{'hex': 'c06a75', 'flight': 'TSC359 ', 'alt_b..."
3,new_adsb,2025-01-31 00:00:00.337039,"{'hex': 'c078ba', 'flight': 'TSC939 ', 'alt_b..."
4,new_adsb,2025-01-31 00:00:00.337196,"{'hex': 'c027da', 'flight': 'CJT990 ', 'alt_b..."


In [15]:
pandas_df = pd.json_normalize(df_pandas['payload'])
print(pandas_df.shape)

(1829647, 41)


## Normalising the dataset took around 50 secs

In [16]:
pandas_df.head()

,hex,flight,alt_baro,alt_geom,gs,track,baro_rate,squawk,emergency,category,...,nav_modes,geom_rate,nav_altitude_fms,tas,mag_heading,true_heading,ias,mach,track_rate,roll
0,ab35d3,DAL136,37000,36475.0,552.7,41.3,0.0,5740,none,A5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,c03f37,WJA598,23925,23325.0,352.3,22.7,-960.0,0512,none,A3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,c06a75,TSC359,24725,24125.0,482.7,47.5,-1856.0,1541,none,A3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,c078ba,TSC939,15175,14600.0,420.6,48.1,-2944.0,6107,none,A5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,c027da,CJT990,30000,29450.0,343.8,229.1,0.0,0630,none,A5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
pandas_df = df_pandas[['dt']].join(pandas_df)
pandas_df.head()

,dt,hex,flight,alt_baro,alt_geom,gs,track,baro_rate,squawk,emergency,...,nav_modes,geom_rate,nav_altitude_fms,tas,mag_heading,true_heading,ias,mach,track_rate,roll
0,2025-01-31 00:00:00.330843,ab35d3,DAL136,37000,36475.0,552.7,41.3,0.0,5740,none,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2025-01-31 00:00:00.336607,c03f37,WJA598,23925,23325.0,352.3,22.7,-960.0,0512,none,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2025-01-31 00:00:00.336872,c06a75,TSC359,24725,24125.0,482.7,47.5,-1856.0,1541,none,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2025-01-31 00:00:00.337039,c078ba,TSC939,15175,14600.0,420.6,48.1,-2944.0,6107,none,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2025-01-31 00:00:00.337196,c027da,CJT990,30000,29450.0,343.8,229.1,0.0,0630,none,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
print("This dataset has {} rows and {} columns".format(pandas_df.shape[0], pandas_df.shape[1]))

This dataset has 1829647 rows and 42 columns


## Trying the same with PySpark

In [2]:
spark = SparkSession.builder \
    .appName("MyApp") \
    .master("local[*]") \
    .config("spark.ui.port", "4050") \
    .getOrCreate()


In [8]:
## Boiler code to start spark session with APP name " LoadExtractedData " and Master "local[*]" with 4GB of memory 
## Rest other parameters are default as we are running in local mode
from pyspark.sql import SparkSession    

from pyspark.sql import SparkSession    

spark = SparkSession.builder \
    .appName("LoadExtractedData") \
    .master("local[*]") \
    .config("spark.executor.memory", "4g") \
    .getOrCreate()


df_spark = spark.read.json("D:\Project\Spark_project\combined_file.txt")
df_spark_large = spark.read.json("D:\Project\Spark_project\combined_file1.txt")


<>:14: SyntaxWarning: invalid escape sequence '\P'
<>:15: SyntaxWarning: invalid escape sequence '\P'
<>:14: SyntaxWarning: invalid escape sequence '\P'
<>:15: SyntaxWarning: invalid escape sequence '\P'
C:\Users\raksh\AppData\Local\Temp\ipykernel_22596\2579727867.py:14: SyntaxWarning: invalid escape sequence '\P'
  df_spark = spark.read.json("D:\Project\Spark_project\combined_file.txt")
C:\Users\raksh\AppData\Local\Temp\ipykernel_22596\2579727867.py:15: SyntaxWarning: invalid escape sequence '\P'
  df_spark_large = spark.read.json("D:\Project\Spark_project\combined_file1.txt")


## Here we can see that Spark has taken 25seconds to load 1 GB file as well as 10 GB file

In [6]:
SparkSession.getActiveSession() and SparkSession.getActiveSession().stop()

In [9]:
print("Count in the dataframe is ",df_spark.count())
print("Count in the dataframe is ",df_spark_large.count())

Count in the dataframe is  1829647
Count in the dataframe is  15509971


In [16]:
print("Schema of the dataframe is ",df_spark.printSchema())

root
 |-- dt: string (nullable = true)
 |-- payload: struct (nullable = true)
 |    |-- alt_baro: string (nullable = true)
 |    |-- alt_geom: long (nullable = true)
 |    |-- baro_rate: long (nullable = true)
 |    |-- category: string (nullable = true)
 |    |-- emergency: string (nullable = true)
 |    |-- flight: string (nullable = true)
 |    |-- geom_rate: long (nullable = true)
 |    |-- gs: double (nullable = true)
 |    |-- gva: long (nullable = true)
 |    |-- hex: string (nullable = true)
 |    |-- ias: long (nullable = true)
 |    |-- lat: double (nullable = true)
 |    |-- lon: double (nullable = true)
 |    |-- mach: double (nullable = true)
 |    |-- mag_heading: double (nullable = true)
 |    |-- messages: long (nullable = true)
 |    |-- mlat: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- nac_p: long (nullable = true)
 |    |-- nac_v: long (nullable = true)
 |    |-- nav_altitude_fms: long (nullable = true)
 |    |-- nav_altitud

In [1]:
from pyspark.sql.functions import col, to_date

flattened_df = df_spark.select(
    col("type").alias("Type"),
    col("dt").alias("DT"),
    col("payload.hex").alias("HexID"),
    col("payload.flight").alias("FlightNumber"),
    col("payload.alt_baro").alias("AltBaro"),
    col("payload.alt_geom").alias("AltGeom"),
    col("payload.gs").alias("GroundSpeed"),
    col("payload.track").alias("Track"),
    col("payload.baro_rate").alias("BaroRate"),
    col("payload.squawk").alias("Squawk"),
    col("payload.emergency").alias("Emergency"),
    col("payload.category").alias("Category"),
    col("payload.nav_qnh").alias("NavQNH"),
    col("payload.nav_altitude_mcp").alias("NavAltitudeMCP"),
    col("payload.nav_heading").alias("NavHeading"),
    col("payload.lat").alias("Latitude"),
    col("payload.lon").alias("Longitude"),
    col("payload.nic").alias("NIC"),
    col("payload.rc").alias("RC"),
    col("payload.seen_pos").alias("SeenPos"),
    col("payload.version").alias("Version"),
    col("payload.nic_baro").alias("NICBaro"),
    col("payload.nac_p").alias("NAC_P"),
    col("payload.nac_v").alias("NAC_V"),
    col("payload.sil").alias("SIL"),
    col("payload.sil_type").alias("SIL_Type"),
    col("payload.gva").alias("GVA"),
    col("payload.sda").alias("SDA"),
    col("payload.mlat").alias("MLAT"),
    col("payload.tisb").alias("TISB"),
    col("payload.messages").alias("Messages"),
    col("payload.seen").alias("Seen"),
    col("payload.rssi").alias("RSSI")
)

flattened_df.printSchema()
flattened_df.show(5, truncate=False)




NameError: name 'df_spark' is not defined

In [39]:
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, ArrayType, FloatType

# Define a schema for the nested payload
payload_schema = StructType([
        StructField("hex", StringType(), True),
        StructField("flight", StringType(), True),
        StructField("alt_baro", IntegerType(), True),
        StructField("alt_geom", IntegerType(), True),
        StructField("gs", FloatType(), True),
        StructField("track", FloatType(), True),
        StructField("baro_rate", IntegerType(), True),
        StructField("squawk", StringType(), True),
        StructField("emergency", StringType(), True),
        StructField("category", StringType(), True),
        StructField("nav_qnh", FloatType(), True),
        StructField("nav_altitude_mcp", IntegerType(), True),
        StructField("nav_heading", FloatType(), True),
        StructField("lat", FloatType(), True),
        StructField("lon", FloatType(), True),
        StructField("nic", IntegerType(), True),
        StructField("rc", IntegerType(), True),
        StructField("seen_pos", FloatType(), True),
        StructField("version", IntegerType(), True),
        StructField("nic_baro", IntegerType(), True),
        StructField("nac_p", IntegerType(), True),
        StructField("nac_v", IntegerType(), True),
        StructField("sil", IntegerType(), True),
        StructField("sil_type", StringType(), True),
        StructField("gva", IntegerType(), True),
        StructField("sda", IntegerType(), True),
        # Assuming mlat and tisb are arrays of strings; adjust as needed
        StructField("mlat", ArrayType(StringType()), True),
        StructField("tisb", ArrayType(StringType()), True),
        StructField("messages", IntegerType(), True),
        StructField("seen", FloatType(), True),
        StructField("rssi", FloatType(), True)
    ])

# Parse the 'payload' string into a struct using the defined schema
df_parsed = df_spark_large.withColumn("payload", from_json(col("payload"), payload_schema))

# Now flatten the JSON by selecting fields from the parsed struct
flattened_df2 = df_parsed.select(
    col("type").alias("Type"),
    col("dt").alias("DT"),
    col("payload.hex").alias("HexID"),
    col("payload.flight").alias("FlightNumber"),
    col("payload.alt_baro").alias("AltBaro"),
    col("payload.alt_geom").alias("AltGeom"),
    col("payload.gs").alias("GroundSpeed"),
    col("payload.track").alias("Track"),
    col("payload.baro_rate").alias("BaroRate"),
    col("payload.squawk").alias("Squawk"),
    col("payload.emergency").alias("Emergency"),
    col("payload.category").alias("Category"),
    col("payload.nav_qnh").alias("NavQNH"),
    col("payload.nav_altitude_mcp").alias("NavAltitudeMCP"),
    col("payload.nav_heading").alias("NavHeading"),
    col("payload.lat").alias("Latitude"),
    col("payload.lon").alias("Longitude"),
    col("payload.nic").alias("NIC"),
    col("payload.rc").alias("RC"),
    col("payload.seen_pos").alias("SeenPos"),
    col("payload.version").alias("Version"),
    col("payload.nic_baro").alias("NICBaro"),
    col("payload.nac_p").alias("NAC_P"),
    col("payload.nac_v").alias("NAC_V"),
    col("payload.sil").alias("SIL"),
    col("payload.sil_type").alias("SIL_Type"),
    col("payload.gva").alias("GVA"),
    col("payload.sda").alias("SDA"),
    col("payload.mlat").alias("MLAT"),
    col("payload.tisb").alias("TISB"),
    col("payload.messages").alias("Messages"),
    col("payload.seen").alias("Seen"),
    col("payload.rssi").alias("RSSI")
    )

flattened_df2.printSchema()
flattened_df2.show(5, truncate=False)


root
 |-- Type: string (nullable = true)
 |-- DT: string (nullable = true)
 |-- HexID: string (nullable = true)
 |-- FlightNumber: string (nullable = true)
 |-- AltBaro: integer (nullable = true)
 |-- AltGeom: integer (nullable = true)
 |-- GroundSpeed: float (nullable = true)
 |-- Track: float (nullable = true)
 |-- BaroRate: integer (nullable = true)
 |-- Squawk: string (nullable = true)
 |-- Emergency: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- NavQNH: float (nullable = true)
 |-- NavAltitudeMCP: integer (nullable = true)
 |-- NavHeading: float (nullable = true)
 |-- Latitude: float (nullable = true)
 |-- Longitude: float (nullable = true)
 |-- NIC: integer (nullable = true)
 |-- RC: integer (nullable = true)
 |-- SeenPos: float (nullable = true)
 |-- Version: integer (nullable = true)
 |-- NICBaro: integer (nullable = true)
 |-- NAC_P: integer (nullable = true)
 |-- NAC_V: integer (nullable = true)
 |-- SIL: integer (nullable = true)
 |-- SIL_Type: string

In [40]:
flattened_df2.count()

15509971

In [41]:
flattened_df2.show(5, truncate=False)

+--------+--------------------------+------+------------+-------+-------+-----------+-----+--------+------+---------+--------+------+--------------+----------+---------+----------+---+---+-------+-------+-------+-----+-----+---+--------+---+---+----+----+--------+----+-----+
|Type    |DT                        |HexID |FlightNumber|AltBaro|AltGeom|GroundSpeed|Track|BaroRate|Squawk|Emergency|Category|NavQNH|NavAltitudeMCP|NavHeading|Latitude |Longitude |NIC|RC |SeenPos|Version|NICBaro|NAC_P|NAC_V|SIL|SIL_Type|GVA|SDA|MLAT|TISB|Messages|Seen|RSSI |
+--------+--------------------------+------+------------+-------+-------+-----------+-----+--------+------+---------+--------+------+--------------+----------+---------+----------+---+---+-------+-------+-------+-----+-----+---+--------+---+---+----+----+--------+----+-----+
|new_adsb|2024-11-05 00:00:00.927706|407799|BAW19W      |39000  |40275  |537.8      |66.4 |0       |3405  |none     |A5      |1013.6|39008         |73.1      |44.1487  |-74

In [42]:
new_df2 = flattened_df2.withColumn("date", to_date(col("dt")))



In [43]:
new_df2.show(5, truncate=False)

+--------+--------------------------+------+------------+-------+-------+-----------+-----+--------+------+---------+--------+------+--------------+----------+---------+----------+---+---+-------+-------+-------+-----+-----+---+--------+---+---+----+----+--------+----+-----+----------+
|Type    |DT                        |HexID |FlightNumber|AltBaro|AltGeom|GroundSpeed|Track|BaroRate|Squawk|Emergency|Category|NavQNH|NavAltitudeMCP|NavHeading|Latitude |Longitude |NIC|RC |SeenPos|Version|NICBaro|NAC_P|NAC_V|SIL|SIL_Type|GVA|SDA|MLAT|TISB|Messages|Seen|RSSI |date      |
+--------+--------------------------+------+------------+-------+-------+-----------+-----+--------+------+---------+--------+------+--------------+----------+---------+----------+---+---+-------+-------+-------+-----+-----+---+--------+---+---+----+----+--------+----+-----+----------+
|new_adsb|2024-11-05 00:00:00.927706|407799|BAW19W      |39000  |40275  |537.8      |66.4 |0       |3405  |none     |A5      |1013.6|39008 

In [44]:
new_df2.groupBy("date").count().show()

+----------+------+
|      date| count|
+----------+------+
|2024-11-05|281394|
|2024-11-06|275072|
|2024-11-07|301973|
|      NULL|     3|
|2024-11-08|287635|
|2024-11-09|252140|
|2024-11-10|268101|
|2024-11-11|289233|
|2024-11-12|268744|
|2024-11-13|303106|
|2024-11-14|321924|
|2024-11-15|291571|
|2024-11-16|267633|
|2024-11-17|293928|
|2024-11-18|285666|
|2024-11-19|274995|
|2024-11-20|274723|
|2024-11-21|288035|
|2024-11-22|283428|
|2024-11-23|237516|
+----------+------+
only showing top 20 rows



In [45]:
flattened_df2.columns

['Type',
 'DT',
 'HexID',
 'FlightNumber',
 'AltBaro',
 'AltGeom',
 'GroundSpeed',
 'Track',
 'BaroRate',
 'Squawk',
 'Emergency',
 'Category',
 'NavQNH',
 'NavAltitudeMCP',
 'NavHeading',
 'Latitude',
 'Longitude',
 'NIC',
 'RC',
 'SeenPos',
 'Version',
 'NICBaro',
 'NAC_P',
 'NAC_V',
 'SIL',
 'SIL_Type',
 'GVA',
 'SDA',
 'MLAT',
 'TISB',
 'Messages',
 'Seen',
 'RSSI']

In [46]:
from pyspark.sql.functions import explode, col

# Explode the 'mlat' array so each element appears in its own row
df_exploded = flattened_df2.select("FlightNumber", explode(col("mlat")).alias("mlat_element"))
df_exploded.show(10, truncate=False)


+------------+------------+
|FlightNumber|mlat_element|
+------------+------------+
+------------+------------+



In [48]:
flattened_df2.select("FlightNumber", "mlat").show(10, truncate=False)

+------------+----+
|FlightNumber|mlat|
+------------+----+
|BAW19W      |[]  |
|00000000    |[]  |
|CJT626      |[]  |
|ACA842      |[]  |
|BAW19W      |[]  |
|00000000    |[]  |
|CJT626      |[]  |
|ACA842      |[]  |
|BAW19W      |[]  |
|CJT626      |[]  |
+------------+----+
only showing top 10 rows



In [50]:
from pyspark.sql.functions import size, col

# Filter rows where the "mlat" array has more than 0 elements
non_empty_mlat = flattened_df2.filter(size(col("mlat")) > 0)

# Show the first few rows where "mlat" is not empty
non_empty_mlat.show(10, truncate=False)


+----+---+-----+------------+-------+-------+-----------+-----+--------+------+---------+--------+------+--------------+----------+--------+---------+---+---+-------+-------+-------+-----+-----+---+--------+---+---+----+----+--------+----+----+
|Type|DT |HexID|FlightNumber|AltBaro|AltGeom|GroundSpeed|Track|BaroRate|Squawk|Emergency|Category|NavQNH|NavAltitudeMCP|NavHeading|Latitude|Longitude|NIC|RC |SeenPos|Version|NICBaro|NAC_P|NAC_V|SIL|SIL_Type|GVA|SDA|MLAT|TISB|Messages|Seen|RSSI|
+----+---+-----+------------+-------+-------+-----------+-----+--------+------+---------+--------+------+--------------+----------+--------+---------+---+---+-------+-------+-------+-----+-----+---+--------+---+---+----+----+--------+----+----+
+----+---+-----+------------+-------+-------+-----------+-----+--------+------+---------+--------+------+--------------+----------+--------+---------+---+---+-------+-------+-------+-----+-----+---+--------+---+---+----+----+--------+----+----+

